In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
from time import perf_counter
import torch
from gnn_scheduler.utils import get_data_path
from torch_geometric.data import HeteroData, InMemoryDataset  # type: ignore[import-untyped]
from torch_geometric.data.storage import (  # type: ignore[import-untyped]
    BaseStorage,
    NodeStorage,
    EdgeStorage,
)
from torch_geometric.data.collate import collate
from gnn_scheduler.data import (
    JobShopData,
)

torch.serialization.add_safe_globals(
    [
        JobShopData,
        HeteroData,
        BaseStorage,
        NodeStorage,
        EdgeStorage,
    ]
)

In [31]:
filename = "instances10x10_eval_0.pt"
DATA_PATH = get_data_path()
pt_file = DATA_PATH / "processed" / filename

In [32]:
start_time = perf_counter()
data = torch.load(pt_file, weights_only=False)
total_time = perf_counter() - start_time
print(f"Loading time: {total_time:.2f} seconds")

Loading time: 97.88 seconds


In [ ]:
class JobShopInMemoryDataset(InMemoryDataset):

    def __init__(self, root, transform=None, pre_transform=None, filename: str = ""):
        super().__init__(root, transform, pre_transform)
        torch.serialization.add_safe_globals(
            [
                JobShopData,
                HeteroData,
                BaseStorage,
                NodeStorage,
                EdgeStorage,
            ]
        )
        self.filename = filename
        start_time = perf_counter()
        self.load(self.processed_paths[0], data_cls=JobShopData)
        total_time = perf_counter() - start_time
        print(f"Loading time: {total_time:.2f} seconds")

    @property
    def raw_file_names(self):
        return [self.filename]

    @property
    def processed_file_names(self):
        return [f"{self.filename}_processed.pt"]

    def process(self):
        data_list = torch.load(self.raw_paths[0], weights_only=False)
        self.save(data_list, self.processed_paths[0])

In [39]:
in_memory_dataset = JobShopInMemoryDataset(DATA_PATH)

Processing...
Done!


Loading time: 2.58 seconds


In [ ]:
from torch_geometric.loader import DataLoader

loader = DataLoader(in_memory_dataset, batch_size=64, shuffle=True)

i = 0
mx = 10
for batch in loader:
    print(len(batch.y))
    i += 1
    if i >= mx:
        break
    

280
290
281
285
288
289
285
305
288
286
